# Scan world aerosols during one year and look to different kinds of aerosols

- Author :Sylvie Dagoret-Campagne
- affiliation : LAL,IN2P3,CNRS
- organization : LSST 

- creation : Monday 25th April 2016

In [97]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.basemap import Basemap
from matplotlib import colors
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import h5py
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [98]:
year_number=2007   # choose the month number
month_start=0      # choose first month
month_stop=11      # choose last month
month_numbers=np.arange(month_start,month_stop+1)
month_stringnumber=['01','02','03','04','05','06','07','08','09','10','11','12']
number_of_monthes=month_numbers.shape[0]

In [99]:
path_root='/Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/CALIPSO/DATA/ICARE/CALIOP/L3/CAL_LID_L3_APro_AllSky.v3.00/CAL_LID_L3_APro_AllSky.v3.00'
fullpath_root=os.path.join(os.path.join(path_root,str(year_number)),'hdf5')

In [100]:
filename_base='CAL_LID_L3_APro_AllSky-Standard-V3-00' # 

### Specify Input Filenames
- day-  : CAL_LID_L3_APro_AllSky-Standard-V3-00.2006-06D.h5
- night : CAL_LID_L3_APro_AllSky-Standard-V3-00.2006-06N.h5

In [101]:
dayfile_extension= [ str(year_number)+'-'+month_stringnumber[ix]+'D'+'.h5' for ix in month_numbers]

In [102]:
nightfile_extension= [ str(year_number)+'-'+month_stringnumber[ix]+'N'+'.h5' for ix in month_numbers]

In [103]:
dayfiles_fullname=[os.path.join(fullpath_root,filename_base+'.'+dayfile_extension[ix]) for ix in month_numbers ]

In [104]:
nightfiles_fullname=[os.path.join(fullpath_root, filename_base+'.'+nightfile_extension[ix]) for ix in month_numbers]

In [105]:
# /Users/dagoret-campagnesylvie/MacOsX/LSST/MyWork/CALIPSO/DATA/ICARE/CALIOP/L3/CAL_LID_L3_APro_AllSky.v3.00/CAL_LID_L3_APro_AllSky.v3.00/2015/hdf5
#dayfiles_fullname

### defines pdf output filename containing plots

In [106]:
filename_pdf='all_aerosol_worldmap_year_'+str(year_number)+'.pdf'

In [107]:
pp = PdfPages(filename_pdf)

### LSST site

In [108]:
#LSST site
Longitude_lsst = -70.7366833333333 # deg
Latitude_lsst = -30.240741666666672 #deg
Altitude_lsst = 2749.999999999238 #m

## work on first file, reading the data and showing the map for a first test


It set also variables usefull for the loop

In [109]:
fileindex=0

In [110]:
h5f = h5py.File(dayfiles_fullname[fileindex], "r")  # file on which one works

In [111]:
longitude=h5f['Longitude_Midpoint']   # shape =(1,72)
all_longitude=longitude[0,:]    # coordinates for X

In [112]:
latitude=h5f['Latitude_Midpoint']     # shape = (1,85)
all_latitude=latitude[0,:]      # coordinate for Y

In [113]:
X,Y=np.meshgrid(all_longitude,all_latitude) # defines the mesh for the map

In [114]:
aod_mean=h5f['AOD_Mean']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
aod_arr=np.array(aod_mean)  # convert in a numpy array
aod_mean_clean=np.where(np.logical_and(aod_arr>0,aod_arr<1),aod_arr, 0 )  # clean the aod from creazy data    

In [115]:
#all aerosols
#------------
aod_mean=h5f['AOD_Mean']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
aod_shape=aod_mean.shape    # get the original size of the array
aod_arr=np.array(aod_mean)  # convert in a numpy array
aod_mean_clean=np.where(np.logical_and(aod_arr>0,aod_arr<1),aod_arr, 0 )  # clean the aod from creazy data

# dust aerosols
#--------------
aod_dust=h5f['AOD_Mean_Dust']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
dust_arr=np.array(aod_dust)  # convert in a numpy array
aod_dust_clean=np.where(np.logical_and(dust_arr>0,dust_arr<1),dust_arr, 0 )  # clean the aod from creazy data

# dust polluted aerosols
#-----------------------
aod_polluteddust=h5f['AOD_Mean_Polluted_Dust']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
polluteddust_arr=np.array(aod_polluteddust)  # convert in a numpy array
aod_polluteddust_clean=np.where(np.logical_and(polluteddust_arr>0,polluteddust_arr<1),polluteddust_arr, 0 )  # clean the aod from creazy data
 
# smoke aerosols
#----------------
aod_smoke=h5f['AOD_Mean_Smoke']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
smoke_arr=np.array(aod_smoke)  # convert in a numpy array
aod_smoke_clean=np.where(np.logical_and(smoke_arr>0,smoke_arr<1),smoke_arr, 0 )  # clean the aod from creazy data


In [116]:
h5f.close()

In [117]:
fig=plt.figure(figsize=(18,9))
#fig.subplots_adjust(wspace='None',hspace='None',left='None',right='None',top='None',bottom='None')


ax1=fig.add_subplot(2,2,1)
map = Basemap()
map.drawcoastlines(color="white")
map.drawcountries(color="black")
img=map.contourf(X, Y, aod_mean_clean,100,cmap='hot')
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
#plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
title = "All Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[fileindex])
ax1.set_title(title)
#ax1.savefig(pp, format='pdf')
ax1.plot()

ax2=fig.add_subplot(2,2,2)
map = Basemap()
map.drawcoastlines(color="white")
map.drawcountries(color="black")
img=map.contourf(X, Y, aod_dust_clean,100,cmap='hot')
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
#plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
title = "Dust Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[fileindex])
ax2.set_title(title)
#ax2.savefig(pp, format='pdf')
ax2.plot()


ax3=fig.add_subplot(2,2,3)
map = Basemap()
map.drawcoastlines(color="white")
map.drawcountries(color="black")
img=map.contourf(X, Y, aod_polluteddust_clean,100,cmap='hot')
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
#plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
title = "Polluted dust Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[fileindex])
ax3.set_title(title)
#ax3.savefig(pp, format='pdf')
ax3.plot()

ax4=fig.add_subplot(2,2,4)
map = Basemap()
map.drawcoastlines(color="white")
map.drawcountries(color="black")
img=map.contourf(X, Y, aod_smoke_clean,100,cmap='hot')
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
#plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
title = "Smoke Aerosol Optical Depth  Year %d Month %s Day time" % (year_number, month_stringnumber[fileindex])
ax4.set_title(title)
#ax4.savefig(pp, format='pdf')
ax4.plot()

figfilename="Images/WORLDMAPSAODY%d%sDay.pdf" %(year_number,month_stringnumber[fileindex])
plt.savefig(figfilename, bbox_inches='tight')
plt.close(fig)



## Loop to Open files, reading the data and showing the map

In [118]:
for imonth in month_numbers:
    
    
    message= ">>>> Year %d Month %s Day File %s : " %(year_number,month_stringnumber[imonth],dayfiles_fullname[imonth])
    
    
    
    # Start with day time
    #---------------------
    h5fday = h5py.File(dayfiles_fullname[imonth], "r")  # file on which one works
    aod_mean_day=h5fday['AOD_Mean']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    aod_arr_day=np.array(aod_mean_day)  # convert in a numpy array
    aod_mean_clean_day=np.where(np.logical_and(aod_arr_day>0,aod_arr_day<1),aod_arr_day, 0 )  # clean the aod from creazy data 
    
    #all aerosols
    #------------
    aod_mean=h5fday['AOD_Mean']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    aod_arr=np.array(aod_mean)  # convert in a numpy array
    aod_mean_clean_day=np.where(np.logical_and(aod_arr>0,aod_arr<1),aod_arr, 0 )  # clean the aod from creazy data

    # dust aerosols
    #--------------
    aod_dust=h5fday['AOD_Mean_Dust']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    dust_arr=np.array(aod_dust)  # convert in a numpy array
    aod_dust_clean_day=np.where(np.logical_and(dust_arr>0,dust_arr<1),dust_arr, 0 )  # clean the aod from creazy data

    # dust polluted aerosols
    #-----------------------
    aod_polluteddust=h5fday['AOD_Mean_Polluted_Dust']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    polluteddust_arr=np.array(aod_polluteddust)  # convert in a numpy array
    aod_polluteddust_clean_day=np.where(np.logical_and(polluteddust_arr>0,polluteddust_arr<1),polluteddust_arr, 0 )  # clean the aod from creazy data
 
    # smoke aerosols
    #----------------
    aod_smoke=h5fday['AOD_Mean_Smoke']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    smoke_arr=np.array(aod_smoke)  # convert in a numpy array
    aod_smoke_clean_day=np.where(np.logical_and(smoke_arr>0,smoke_arr<1),smoke_arr, 0 )  # clean the aod from creazy data

    #close input file
    h5fday.close()
    
    
    # Start with night time
    #---------------------
    
    message= ">>>> Year %d Month %s Night File %s : " %(year_number,month_stringnumber[imonth],nightfiles_fullname[imonth])
    
    h5fnight = h5py.File(nightfiles_fullname[imonth], "r")  # file on which one works

    
    #all aerosols
    #------------
    aod_mean=h5fnight['AOD_Mean']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    aod_arr=np.array(aod_mean)  # convert in a numpy array
    aod_mean_clean_night=np.where(np.logical_and(aod_arr>0,aod_arr<1),aod_arr, 0 )  # clean the aod from creazy data

    # dust aerosols
    #--------------
    aod_dust=h5fnight['AOD_Mean_Dust']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    dust_arr=np.array(aod_dust)  # convert in a numpy array
    aod_dust_clean_night=np.where(np.logical_and(dust_arr>0,dust_arr<1),dust_arr, 0 )  # clean the aod from creazy data

    # dust polluted aerosols
    #-----------------------
    aod_polluteddust=h5fnight['AOD_Mean_Polluted_Dust']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    polluteddust_arr=np.array(aod_polluteddust)  # convert in a numpy array
    aod_polluteddust_clean_night=np.where(np.logical_and(polluteddust_arr>0,polluteddust_arr<1),polluteddust_arr, 0 )  # clean the aod from creazy data
 
    # smoke aerosols
    #----------------
    aod_smoke=h5fnight['AOD_Mean_Smoke']  # the data table we want to explore is the Aerosol Optical Depth
                          # 85 rows in latitude, 70 columns in longitude
    smoke_arr=np.array(aod_smoke)  # convert in a numpy array
    aod_smoke_clean_night=np.where(np.logical_and(smoke_arr>0,smoke_arr<1),smoke_arr, 0 )  # clean the aod from creazy data 
    
    
    h5fnight.close()
  


    # day map for this month
    #------------------------

    fig=plt.figure(figsize=(18,9))
    #fig.subplots_adjust(wspace='None',hspace='None',left='None',right='None',top='None',bottom='None')


    ax1=fig.add_subplot(2,2,1)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_mean_clean_day,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "All Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[imonth])
    ax1.set_title(title)
    #ax1.savefig(pp, format='pdf')
    ax1.plot()

    ax2=fig.add_subplot(2,2,2)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_dust_clean_day,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "Dust Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[imonth])
    ax2.set_title(title)
    #ax2.savefig(pp, format='pdf')
    ax2.plot()


    ax3=fig.add_subplot(2,2,3)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_polluteddust_clean_day,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "Polluted dust Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[imonth])
    ax3.set_title(title)
    #ax3.savefig(pp, format='pdf')
    ax3.plot()

    ax4=fig.add_subplot(2,2,4)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_smoke_clean_day,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "Smoke Aerosol Optical Depth  Year %s Month %s Day time" % (year_number, month_stringnumber[imonth])
    ax4.set_title(title)
    #ax4.savefig(pp, format='pdf')
    ax4.plot()
    
    figfilename="Images/WORLDMAPSAODY%d%sDay.pdf" %(year_number,month_stringnumber[imonth])
    plt.savefig(figfilename, bbox_inches='tight')
    plt.close(fig)
    #fig.close()
    
    
     # night map for this month
    #------------------------

    fig=plt.figure(figsize=(18,9))
    #fig.subplots_adjust(wspace='None',hspace='None',left='None',right='None',top='None',bottom='None')


    ax1=fig.add_subplot(2,2,1)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_mean_clean_night,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "All Aerosol Optical Depth  Year %s Month %s Night time" % (year_number, month_stringnumber[imonth])
    ax1.set_title(title)
    #ax1.savefig(pp, format='pdf')
    ax1.plot()

    ax2=fig.add_subplot(2,2,2)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_dust_clean_night,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "Dust Aerosol Optical Depth  Year %s Month %s Night time" % (year_number, month_stringnumber[imonth])
    ax2.set_title(title)
    #ax2.savefig(pp, format='pdf')
    ax2.plot()


    ax3=fig.add_subplot(2,2,3)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_polluteddust_clean_night,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "Polluted dust Aerosol Optical Depth  Year %s Month %s Night time" % (year_number, month_stringnumber[imonth])
    ax3.set_title(title)
    #ax3.savefig(pp, format='pdf')
    ax3.plot()

    ax4=fig.add_subplot(2,2,4)
    map = Basemap()
    map.drawcoastlines(color="white")
    map.drawcountries(color="black")
    img=map.contourf(X, Y, aod_smoke_clean_night,100,cmap='hot')
    map.colorbar(img,"right", size="5%", pad="2%")
    map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
    map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
    xpt_lsst,ypt_lsst = map(Longitude_lsst,Latitude_lsst)
    map.plot(xpt_lsst,ypt_lsst,'bo',markersize=10)  # plot a red dot there
    #plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
    title = "Smoke Aerosol Optical Depth  Year %s Month %s Night time" % (year_number, month_stringnumber[imonth])
    ax4.set_title(title)
    #ax4.savefig(pp, format='pdf')
    ax4.plot()
    
    
    figfilename="Images/WORLDMAPSAODY%d%sNight.pdf" %(year_number,month_stringnumber[imonth])
    plt.savefig(figfilename, bbox_inches='tight')
    plt.close(fig)
    #fig.close()
    
    
    
    
    

In [119]:
pp.close()